In [1]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
import load_and_preprocess_data as load
from pprint import pprint
import csv

In [2]:
data = pd.read_csv('../data/diabetes.csv')

In [3]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [4]:
data = load.get_label_appended_data(data)

In [5]:
dataset = []
for i in range(len(data)):
    dataset.append([data.iloc[i, j] for j in range(len(data.columns))])

In [6]:
te = TransactionEncoder()

In [7]:
te_ary = te.fit(dataset).transform(dataset)

In [8]:
te_ary

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False]])

In [9]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [10]:
df

,"Age,21","Age,22","Age,23","Age,24","Age,25","Age,26","Age,27","Age,28","Age,29","Age,30",...,"SkinThickness,50","SkinThickness,51","SkinThickness,52","SkinThickness,54","SkinThickness,56","SkinThickness,60","SkinThickness,63","SkinThickness,7","SkinThickness,8","SkinThickness,99"
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
764,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
765,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
766,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Frequent Itemset Generation

In [11]:
from mlxtend.frequent_patterns import apriori

In [12]:
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)

In [13]:
frequent_itemsets

,support,itemsets
0,0.486979,"(Insulin,0)"
1,0.651042,"(Outcome,0)"
2,0.348958,"(Outcome,1)"
3,0.295573,"(SkinThickness,0)"
4,0.307292,"(Insulin,0, Outcome,0)"
5,0.295573,"(SkinThickness,0, Insulin,0)"


# Association Rule Mining

In [14]:
from mlxtend.frequent_patterns import association_rules

In [15]:
res = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)

In [16]:
res

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Insulin,0)","(Outcome,0)",0.486979,0.651042,0.307292,0.631016,0.969241,-0.009752,0.945728
1,"(Outcome,0)","(Insulin,0)",0.651042,0.486979,0.307292,0.472000,0.969241,-0.009752,0.971630
2,"(SkinThickness,0)","(Insulin,0)",0.295573,0.486979,0.295573,1.000000,2.053476,0.151635,inf
3,"(Insulin,0)","(SkinThickness,0)",0.486979,0.295573,0.295573,0.606952,2.053476,0.151635,1.792216
